### Final Code & Debug 

In [4]:
from decimal import Decimal, ROUND_HALF_UP, getcontext
from typing import Any, Tuple

getcontext().prec = 28

def decimal(value):
    if isinstance(value, Decimal):
        return value
    try:
        text_version = str(value)
        return Decimal(text_version)
    except Exception as exc:
        raise TypeError(f"The amount has to be a number, got {type(value).__name__}") from exc

def format_dec(value):
    return str(value.quantize(Decimal("0.01"), rounding=ROUND_HALF_UP))

class BankAccount:
    def __init__(self, account_holder: str, balance: Any = "0.00") -> None:
        self.account_holder = account_holder
        self.balance = decimal(balance)

    def deposit(self, amount: Any) -> Decimal:
        amount_to_add = decimal(amount)
        if amount_to_add <= 0:
            raise ValueError("Amount must be greater than 0")  
        current_balance = self.balance
        new_balance = current_balance + amount_to_add
        self.balance = new_balance
        return self.balance

    def withdraw(self, amount: Any) -> Tuple[bool, Decimal]:
        amount_to_withdraw = decimal(amount)
        if amount_to_withdraw <= 0:
            raise ValueError("Amount must be greater than 0")
        if self.balance < amount_to_withdraw:
            print("Insufficient funds")
            return False, self.balance
        current_balance = self.balance
        updated_balance = current_balance - amount_to_withdraw
        self.balance = updated_balance
        return True, self.balance

    def account_info(self) -> str:
        nice_balance = format_dec(self.balance)
        return f"Account holder: {self.account_holder}, Balance: {nice_balance}"
    
class SavingsAccount(BankAccount):
    def __init__(self, account_holder: str, balance: Any = "0.00", interest_rate: Any = "0.02") -> None:
        super().__init__(account_holder, balance)
        self.interest_rate = decimal(interest_rate)

    def apply_interest(self) -> Decimal:
        one = Decimal("1")
        growth_factor = one + self.interest_rate
        current_balance = self.balance
        new_balance = current_balance * growth_factor
        self.balance = new_balance
        return self.balance

class CheckingAccount(BankAccount):
    def __init__(self, account_holder: str, balance: Any = "0.00", transaction_fee: Any = "1.00") -> None:
        super().__init__(account_holder, balance)
        self.transaction_fee = decimal(transaction_fee)

    def withdraw(self, amount: Any) -> Tuple[bool, Decimal]:
        total = decimal(amount) + self.transaction_fee
        return super().withdraw(total)


In [5]:
b1 = BankAccount("Test 15", 0)

try:
    b1.deposit(object())
except Exception as e:
    print("If error:", type(e).__name__, "-", e)

try:
    b1.deposit(object(abc))
except Exception as e:
    print("If error:", type(e).__name__, "-", e)

try:
    b1.withdraw(-5)
except Exception as e:
    print("withdraw(-5):", type(e).__name__, "-", e)

try:
    b1.deposit(0)
except Exception as e:
    print("deposit(0) error:", type(e).__name__, "-", e)

If error: TypeError - The amount has to be a number, got object
If error: NameError - name 'abc' is not defined
withdraw(-5): ValueError - Amount must be greater than 0
deposit(0) error: ValueError - Amount must be greater than 0


In [6]:
k1 = BankAccount("Test 1", 0)
print("Start:", k1.account_info())            
k1.deposit(175)
print("After deposit 100:", k1.account_info())
ok, _ = k1.withdraw(26)
print("Withdraw 50 ok?:", ok)
print("After withdraw:", k1.account_info())

# savings account fixed interest 2%
s1 = SavingsAccount("Test 2", 100)          
print("Before:", s1.account_info())         
s1.apply_interest()
print("After apply_interest:", s1.account_info()) 

# savings account custom interest with 15%
s2 = SavingsAccount("Test 3", 200, interest_rate="0.15")
print("Before:", s2.account_info())           
s2.apply_interest()
print("After apply_interest:", s2.account_info())

# withdraw 10 with fee  1 
c1 = CheckingAccount("Test 4", 100, transaction_fee="1.00")
print("Start:", c1.account_info())          
ok, _ = c1.withdraw(10)                      
print("Withdraw 10 authorization:", ok)
print("After:", c1.account_info())  

# fee and over-withdraw
c2 = CheckingAccount("Test 5", 10, transaction_fee="1")
print("Start:", c2.account_info())         
ok, _ = c2.withdraw(10)                       
print("Withdraw 10 authorization:", ok)
print(c2.account_info())

Start: Account holder: Test 1, Balance: 0.00
After deposit 100: Account holder: Test 1, Balance: 175.00
Withdraw 50 ok?: True
After withdraw: Account holder: Test 1, Balance: 149.00
Before: Account holder: Test 2, Balance: 100.00
After apply_interest: Account holder: Test 2, Balance: 102.00
Before: Account holder: Test 3, Balance: 200.00
After apply_interest: Account holder: Test 3, Balance: 230.00
Start: Account holder: Test 4, Balance: 100.00
Withdraw 10 authorization: True
After: Account holder: Test 4, Balance: 89.00
Start: Account holder: Test 5, Balance: 10.00
Insufficient funds
Withdraw 10 authorization: False
Account holder: Test 5, Balance: 10.00


### Assignment Step by Step

### 1. Design and Implement the Class Structure:
##### o Create a base class BankAccount with the attributes and methods specified above.
##### o Create derived classes SavingsAccount and CheckingAccount that inherit from BankAccount and implement additional functionality specific to each type of account.

1. Base Class: BankAccount
###### o Attributes: 
###### account_holder: Name of the account holder.
###### alance: The current balance (initialized to 0 by default).
###### o Methods:
###### deposit(amount): Adds the specified amount to the account balance.
###### withdraw(amount): Subtracts the specified amount from the balance, if sufficient funds exist. If not, print a message indicating insufficient funds. 
###### account_info(): Returns the account holder's name and current balance.

In [7]:
from decimal import Decimal, ROUND_HALF_UP, getcontext
from typing import Any, Tuple

In [8]:
getcontext().prec = 28

In [9]:
def decimal(value):
    if isinstance(value, Decimal):
        return value
    try:
        text_version = str(value)
        return Decimal(text_version)
    except Exception as exc:
        raise TypeError(f"The amount has to be a number, got {type(value).__name__}") from exc

def format_dec(value):
    return str(value.quantize(Decimal("0.01"), rounding=ROUND_HALF_UP))

In [10]:
class BankAccount:
    def __init__(self, account_holder: str, balance: Any = "0.00") -> None:
        self.account_holder = account_holder
        self.balance = decimal(balance)

    def deposit(self, amount: Any) -> Decimal:
        amount_to_add = decimal(amount)
        if amount_to_add <= 0:
            raise ValueError("Amount must be greater than 0") 
        current_balance = self.balance
        new_balance = current_balance + amount_to_add
        self.balance = new_balance
        return self.balance


##### Adding withdrawl in class

In [11]:
class BankAccount:
    def __init__(self, account_holder: str, balance: Any = "0.00") -> None:
        self.account_holder = account_holder
        self.balance = decimal(balance)

    def deposit(self, amount: Any) -> Decimal:
        amount_to_add = decimal(amount)
        if amount_to_add <= 0:
            raise ValueError("Amount must be greater than 0")
        current_balance = self.balance
        new_balance = current_balance + amount_to_add
        self.balance = new_balance
        return self.balance

    def withdraw(self, amount: Any) -> Tuple[bool, Decimal]:
        """Take money out if there is enough (returns (ok, new_balance))."""
        amount_to_withdraw = decimal(amount)
        if amount_to_withdraw <= 0:
            raise ValueError("Amount must be greater than 0")
        # check funds the simple way
        if self.balance < amount_to_withdraw:
            print("Insufficient funds")
            return False, self.balance
        current_balance = self.balance
        updated_balance = current_balance - amount_to_withdraw
        self.balance = updated_balance
        return True, self.balance

##### Adding account info in class

In [12]:
class BankAccount:
    def __init__(self, account_holder: str, balance: Any = "0.00") -> None:
        self.account_holder = account_holder
        self.balance = decimal(balance)

    def deposit(self, amount: Any) -> Decimal:
        amount_to_add = decimal(amount)
        if amount_to_add <= 0:
            raise ValueError("Amount must be greater than 0")  
        current_balance = self.balance
        new_balance = current_balance + amount_to_add
        self.balance = new_balance
        return self.balance

    def withdraw(self, amount: Any) -> Tuple[bool, Decimal]:
        amount_to_withdraw = decimal(amount)
        if amount_to_withdraw <= 0:
            raise ValueError("Amount must be greater than 0")
        if self.balance < amount_to_withdraw:
            print("Insufficient funds")
            return False, self.balance
        current_balance = self.balance
        updated_balance = current_balance - amount_to_withdraw
        self.balance = updated_balance
        return True, self.balance

    def account_info(self) -> str:
        nice_balance = format_dec(self.balance)
        return f"Account holder: {self.account_holder}, Balance: {nice_balance}"

In [13]:
acc = BankAccount("You")
acc.deposit(50)
info = acc.account_info()
print(info)

Account holder: You, Balance: 50.00


2. Derived Class: SavingsAccount
###### o Attributes:
###### interest_rate: A fixed interest rate (e.g., 2% annually).
###### o Methods:
###### apply_interest(): Applies the interest to the balance. (e.g., increases the balance by multiplying it by (1 + interest_rate)).

In [14]:
class SavingsAccount(BankAccount):
    def __init__(self, account_holder: str, balance: Any = "0.00", interest_rate: Any = "0.02") -> None:
        super().__init__(account_holder, balance)
        self.interest_rate = decimal(interest_rate)

    def apply_interest(self) -> Decimal:
        one = Decimal("1")
        growth_factor = one + self.interest_rate
        current_balance = self.balance
        new_balance = current_balance * growth_factor
        self.balance = new_balance
        return self.balance

3. Derived Class: CheckingAccount
###### o Attributes:
###### transaction_fee: A fixed fee (e.g., $1) that is charged for every withdrawal.
###### o Methods:
###### withdraw(amount): This method overrides the base class method to subtract the transaction fee in addition to the withdrawn amount.

In [15]:
class CheckingAccount(BankAccount):
    def __init__(self, account_holder: str, balance: Any = "0.00", transaction_fee: Any = "1.00") -> None:
        super().__init__(account_holder, balance)
        self.transaction_fee = decimal(transaction_fee)

    def withdraw(self, amount: Any) -> Tuple[bool, Decimal]:
        amount_user_wants = decimal(amount)
        if amount_user_wants <= 0:
            raise ValueError("Amount must be greater than 0")
        total_to_subtract = amount_user_wants + self.transaction_fee
        if self.balance < total_to_subtract:
            print("Insufficient funds")
            return False, self.balance
        before = self.balance
        after = before - total_to_subtract
        self.balance = after
        return True, self.balance

### 2. Inheritance and Method Overriding:
##### o Ensure that the withdraw() method in CheckingAccount correctly overrides the withdraw() method from BankAccount.

In [16]:
#wrote a new withdraw in CheckingAccount that charges a fee, 
#so it’s a different function than BankAccount.withdraw. 
#They’re two different functions, comparing them with is (same object?) taht returns False.

print("Same function object?",
      BankAccount.withdraw is CheckingAccount.withdraw)

Same function object? False


In [17]:
ref: BankAccount = CheckingAccount("Tester", balance="100.00", transaction_fee="1.00")
# testing if withdraw 10, it should cost 11 with the fee, 
# it should have balance of 89.00 if override is active
ok, bal = ref.withdraw(10)
print("Call via ref (base) ->", ok, ref.account_info())

Call via ref (base) -> True Account holder: Tester, Balance: 89.00


##### o Ensure that SavingsAccount has a method apply_interest() that applies the interest rate to the balance.

In [18]:
#fixed interest applied test 

s1 = SavingsAccount("You", 100)
print("Before:", s1.account_info())   # 100.00
s1.apply_interest()
print("After: ", s1.account_info())   # 102.00

Before: Account holder: You, Balance: 100.00
After:  Account holder: You, Balance: 102.00


In [19]:
# zero balance edge

s3 = SavingsAccount("zero balance man", 0, interest_rate="0.02")
print("Before:", s3.account_info())   # 0.00
s3.apply_interest()
print("After: ", s3.account_info())   # 0.00

Before: Account holder: zero balance man, Balance: 0.00
After:  Account holder: zero balance man, Balance: 0.00


### 3. Debugging:
##### o Test the classes with various inputs to ensure correct behavior (e.g., deposit 100, withdraw 50, try over-withdrawing, apply interest, etc).

In [20]:
# BankAccount test deposit 175 and withdraw 26

b1 = BankAccount("Test 1", 0)
print("Start:", b1.account_info())            
b1.deposit(175)
print("After deposit 100:", b1.account_info())
ok, _ = b1.withdraw(26)
print("Withdraw 50 ok?:", ok)
print("After withdraw:", b1.account_info())


Start: Account holder: Test 1, Balance: 0.00
After deposit 100: Account holder: Test 1, Balance: 175.00
Withdraw 50 ok?: True
After withdraw: Account holder: Test 1, Balance: 149.00


##### o Ensure that the interest is correctly applied to the balance in SavingsAccount.

In [21]:
# savings account fixed interest 2%
s1 = SavingsAccount("Test 2", 100)          
print("Before:", s1.account_info())         
s1.apply_interest()
print("After apply_interest:", s1.account_info()) 


Before: Account holder: Test 2, Balance: 100.00
After apply_interest: Account holder: Test 2, Balance: 102.00


In [22]:
# savings account custom interest with 15%
s2 = SavingsAccount("Test 3", 200, interest_rate="0.15")
print("Before:", s2.account_info())           
s2.apply_interest()
print("After apply_interest:", s2.account_info())

Before: Account holder: Test 3, Balance: 200.00
After apply_interest: Account holder: Test 3, Balance: 230.00


##### o Ensure that withdrawing funds from a CheckingAccount deducts both the withdrawal amount and the transaction fee.

In [23]:
# withdraw 10 with fee  1 
c1 = CheckingAccount("Test 4", 100, transaction_fee="1.00")
print("Start:", c1.account_info())          
ok, _ = c1.withdraw(10)                      
print("Withdraw 10 authorization:", ok)
print("After:", c1.account_info())  

Start: Account holder: Test 4, Balance: 100.00
Withdraw 10 authorization: True
After: Account holder: Test 4, Balance: 89.00


In [24]:
# fee and over-withdraw
c2 = CheckingAccount("Test 5", 10, transaction_fee="1")
print("Start:", c2.account_info())         
ok, _ = c2.withdraw(10)                       
print("Withdraw 10 authorization:", ok)
print(c2.account_info())

Start: Account holder: Test 5, Balance: 10.00
Insufficient funds
Withdraw 10 authorization: False
Account holder: Test 5, Balance: 10.00


In [25]:
b1 = BankAccount("Test 15", 0)

In [26]:
try:
    b1.deposit(0)
except Exception as e:
    print("deposit(0) error:", type(e).__name__, "-", e)

deposit(0) error: ValueError - Amount must be greater than 0


In [27]:
try:
    b1.withdraw(-5)
except Exception as e:
    print("withdraw(-5):", type(e).__name__, "-", e)

withdraw(-5): ValueError - Amount must be greater than 0


In [28]:
try:
    b1.deposit(object(abc))
except Exception as e:
    print("If error:", type(e).__name__, "-", e)

If error: NameError - name 'abc' is not defined


In [29]:
try:
    b1.deposit(object())
except Exception as e:
    print("If error:", type(e).__name__, "-", e)


If error: TypeError - The amount has to be a number, got object
